# Demo FAC on-demand quicklook

Using SwarmPAL implementation in development. See <https://swarmpal.readthedocs.io/en/latest/guides/fac/intro_fac.html>

This example uses HAPI instead of the VirES API

In [ ]:
import datetime as dt
from hapiclient import hapi
import numpy as np
from swarmpal.io import create_paldata, PalDataItem
from swarmpal.toolboxes import fac

In [ ]:
SERVER = "https://vires.services/hapi"
DATASET = "SW_OPER_MAGA_LR_1B"

## Find latest available time

In [ ]:
meta = hapi(SERVER, DATASET)
latest_available = dt.datetime.fromisoformat(meta["stopDate"])
latest_available

## Fetch latest available day

In [ ]:
start = latest_available - dt.timedelta(days=1)
stop = latest_available
data = create_paldata(
    PalDataItem.from_hapi(
        server=SERVER,
        dataset=DATASET,
        parameters="Latitude,Longitude,Radius,B_NEC,B_NEC_Model",
        start=start.strftime("%Y-%m-%dT%H:%M:%S"),
        stop=stop.strftime("%Y-%m-%dT%H:%M:%S"),
    )
)
# Temporary fix to data labelling
data = data.rename_dims(
    {"B_NEC_dim1": "NEC", "B_NEC_Model_dim1": "NEC"}
)
print(data)

## Apply FAC algorithm

In [ ]:
process = fac.processes.FAC_singlesat(
    config={
        "dataset": DATASET,
        "model_varname": "B_NEC_Model",
        "measurement_varname": "B_NEC",
    },
)
data.swarmpal.apply(process)
print(data)

In [ ]:
data.swarmpal_fac.quicklook()